In [2]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-11-14 13:43:45.768494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 13:43:46.347590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home2/cbisot/miniconda3/envs/amftrack/lib/python3.7/site-packages/cv2/../../lib64:
2023-11-14 13:43:46.347661: I tensorflow/stream_executor/cuda/cudart_stub.cc:29

In [17]:
plates = [
    # '478_20230814', '468_20230809', '447_20230821', '487_20230922',
    #    '492_20230901', '471_20230821',
    # '486_20231009',
    "494_20230908",
    #       '482_20230908',
    #       '495_20231013',
    #       '463_20231013',
    #    '491_20231013',
    #       '481_20231005', '483_20231005',
    # '490_20231003',
    # '485_20230929',
    # "310_20230830"
]
# plates = ["3_20220426", "12_20220502", "16_20220419", "480_20221205"]
# directory_targ = directory_project
directory_targ = os.path.join(directory_scratch, "stitch_temp") + "/"
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
# update_plate_info(directory_targ, local=True)
# all_folders = get_current_folders(directory_targ, local=True)

Analysis_540_20221227_200_Version1
Analysis_16_20220419_200_Version1
Analysis_558_20230109_200_Version1
Analysis_545_20220115_200_Version1
Analysis_3_20220426_200_Version1
Analysis_551_20220102_200_Version1
Analysis_480_20221205_200_Version1
Analysis_12_20220502_200_Version1
Analysis_552_20230111_200_Version1


In [18]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

select = analysis_folders
select["total_path"] = select["total_path"].str.replace(
    "/projects/0/einf914/data/", "/scratch-shared/amftrack/stitch_temp/"
)

/home/cbisot/miniconda3/envs/amftrack/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
analysis_folders

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,fungalC,split,...,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis,InoculationDate
0,Analysis_478_20230814_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20230814_...,478,15,'Carrot','C2','001P100N100C','000P100N','Y',...,"14.08.2023, 12:32:","31.08.2023, 21:47:",200,Analysis_478_20230814_200_Version1/experiment....,Analysis_478_20230814_200_Version1/global_hyph...,Analysis_478_20230814_200_Version1/time_hypha_...,Analysis_478_20230814_200_Version1/time_plate_...,Analysis_478_20230814_200_Version1/global_plat...,"01.01.1970, 01:00:",NaN
1,Analysis_483_20231005_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20231006_...,483,7,'Carrot','C2','001P100N100C','100P100N','Y',...,"06.10.2023, 14:24:","13.10.2023, 12:35:",57,Analysis_483_20231005_200_Version1/experiment....,Analysis_483_20231005_200_Version1/global_hyph...,Analysis_483_20231005_200_Version1/time_hypha_...,Analysis_483_20231005_200_Version1/time_plate_...,Analysis_483_20231005_200_Version1/global_plat...,"01.01.1970, 01:00:",NaN
2,Analysis_494_20230908_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20230908_...,494,16,'Carrot','C2','001P100N100C','100P100N','Y',...,"08.09.2023, 14:42:","25.09.2023, 06:29:",200,Analysis_494_20230908_200_Version1/experiment....,Analysis_494_20230908_200_Version1/global_hyph...,Analysis_494_20230908_200_Version1/time_hypha_...,Analysis_494_20230908_200_Version1/time_plate_...,Analysis_494_20230908_200_Version1/global_plat...,"01.01.1970, 01:00:",nan
3,Analysis_485_20230929_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20230929_...,485,9,'Carrot','C2','001P100N100C','100P100N','Y',...,"29.09.2023, 16:24:","13.10.2023, 12:41:",134,Analysis_485_20230929_200_Version1/experiment....,Analysis_485_20230929_200_Version1/global_hyph...,Analysis_485_20230929_200_Version1/time_hypha_...,Analysis_485_20230929_200_Version1/time_plate_...,Analysis_485_20230929_200_Version1/global_plat...,"01.01.1970, 01:00:",nan
4,Analysis_492_20230901_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20230901_...,492,2,'Carrot','C2','001P100N100C',nan,'Y',...,"01.09.2023, 13:37:","18.09.2023, 04:06:",200,Analysis_492_20230901_200_Version1/experiment....,Analysis_492_20230901_200_Version1/global_hyph...,Analysis_492_20230901_200_Version1/time_hypha_...,Analysis_492_20230901_200_Version1/time_plate_...,Analysis_492_20230901_200_Version1/global_plat...,"01.01.1970, 01:00:",NaN
5,Analysis_487_20230922_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20230926_...,487,34,'Carrot','C2','001P100N100C',nan,'Y',...,"26.09.2023, 19:26:","02.10.2023, 03:18:",17,Analysis_487_20230922_200_Version1/experiment....,Analysis_487_20230922_200_Version1/global_hyph...,Analysis_487_20230922_200_Version1/time_hypha_...,Analysis_487_20230922_200_Version1/time_plate_...,Analysis_487_20230922_200_Version1/global_plat...,"01.01.1970, 01:00:",NaN
6,Analysis_486_20231009_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20231009_...,486,19,'Carrot','C2','001P100N100C','100P100N','Y',...,"09.10.2023, 20:46:","13.10.2023, 13:10:",21,Analysis_486_20231009_200_Version1/experiment....,Analysis_486_20231009_200_Version1/global_hyph...,Analysis_486_20231009_200_Version1/time_hypha_...,Analysis_486_20231009_200_Version1/time_plate_...,Analysis_486_20231009_200_Version1/global_plat...,"01.01.1970, 01:00:",NaN
7,Analysis_482_20230908_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp/20230908_...,482,15,'Carrot','C2','001P100N100C','100P100N','Y',...,"08.09.2023, 14:39:","12.10.2023, 12:12:",199,Analysis_482_20230908_200_Version1/experiment....,Analysis_482_20230908_200_Version1/global_hyph...,Analysis_482_20230908_200_Version1/time_hypha_...,Analysis_482_20230908_200_Version1/time_plate_...,Analysis_482_20230908_200_Version1/global_plat...,"01.01.1970

In [16]:
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
    load_skel,
)

run_info = whole_plate_info
t = 0
select = run_info.loc[run_info["t"] == t]
row = [row for index, row in select.iterrows()][0]
path_exp = f'{directory}{row["path_exp"]}'
exp = pickle.load(open(path_exp, "rb"))
exp.save_location = "/".join(path_exp.split("/")[:-1])
exp.directory = directory
load_study_zone(exp)
load_graphs(exp, directory, indexes=[t], post_process=True)

In [20]:
for t in range(exp.ts):
    exp.load_tile_information(t)

In [17]:
get_length_density_in_region(exp, t, {"i": 0})

FileNotFoundError: No such file: '/projects/0/einf914/data/20230720_2319_Plate13/Img/Img_r01_c05.tif'

In [21]:
exp.image_paths

[['/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c05.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c06.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c07.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c08.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c09.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c10.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r01_c11.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r02_c04.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r02_c05.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r02_c06.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r02_c07.tif',
  '/scratch-shared/amftrack/stitch_temp/20230720_2319_Plate13/Img/Img_r02_c0

In [15]:
time = "6:40:00"
directory = directory_targ
max_ind = 20
incr = 100

list_f = [
    get_num_trunks,
    get_area,
    get_area_separate_connected_components,
    get_num_tips,
    get_num_nodes,
    get_area_study_zone,
    get_num_tips_study_zone,
    get_num_nodes_study_zone,
    get_num_edges,
    get_length_tot,
    get_length_study_zone,
    get_is_out_study,
    get_mean_edge_straight,
    get_spore_volume,
    get_num_spores,
    get_tot_biovolume_study,
    get_tot_biovolume,
]
list_args = [{}] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    select,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3131428


In [19]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.P_regions import *

%load_ext autoreload
%autoreload 2
directory = directory_targ
overwrite = True
load_graphs = True
name_job = "density3"
# list_f = [
#     get_length_tot,
# ]
# list_args = [{}] * len(list_f)
max_ind = 20
incr = 100
num_parallel = 15
time = "1:00:00"
list_f = [get_surface_area_in_region] * 18
list_f += [get_length_density_in_region] * 18
list_args = [{"i": i} for i in range(18)] * 2
# fs = [
#     get_density_in_ring_new,
#     get_density_active_tips_in_ring,
# ]

# list_f = []
# list_args = []

# for f in fs:
#     list_f += [f] * 20

#     list_args += [{"incr": incr, "i": i, "rh_only": True,"max_t" : 100} for i in range(0, 20)]
args = [directory, True, True]
directory_targ = directory
for index, row in select.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        whole_plate_info = whole_plate_info.loc[whole_plate_info["t"].between(0, 200)]

        run_parallel_post(
            "time_plate_post_process_long.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "density_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
            # dependency = True,
        )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


sbatch: WARNING: the fat partition will be phased out. Use the fat_rome and/or fat_genoa partition instead.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 32 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4443407


sbatch: WARNING: the fat partition will be phased out. Use the fat_rome and/or fat_genoa partition instead.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 32 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4443408


...

Submitted batch job 4443419


sbatch: WARNING: the fat partition will be phased out. Use the fat_rome and/or fat_genoa partition instead.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 32 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4443420


sbatch: WARNING: the fat partition will be phased out. Use the fat_rome and/or fat_genoa partition instead.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 32 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4443422


In [1]:
1

1

In [20]:
name_job = "density3"
run_launcher(
    "analysis_uploader_no_upload.py",
    [directory_targ, name_job, -1],
    plates,
    "6:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 CPUs, based on the number of CPUs and the amount memory that you've requested.


Submitted batch job 4443423


In [7]:
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    # get_width_tip_edge,
    # get_width_root_edge,
    # get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127884


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127885


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 3127944


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127945


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 8 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3127946
